In [ ]:
import pandas
from ast import literal_eval
import theano.tensor as T
import numpy as np
import pymc3 as pm
from functools import reduce

df = pandas.read_csv('./processed/segment_6.csv')
for col in df.columns:
    df.loc[:,col] = df.loc[:,col].apply(lambda x: literal_eval(x))

sigmaBG = 20; # TODO: infer
tauBG = 0.095; # SD of log(BG). If 95% obs w/i 20%, then SD ~ 10%. log(1.1) = 0.095.
    # Note: lognormal is a lot slower for inference
sigmaIns = 0.1; # TODO: use, then possibly infer
sigmaCarbs = 5; # TODO: use, then possibly infer

nSeg = len(df['Basal'][0])
nSeq = len(df['Basal'])

basal = [ [df['Basal'][i][j] for i in range(nSeq)]
         for j in range(nSeg) ] # basal[i] is array of all basals at first meas in sequence
smbg  = [ [df['MeasBG'][i][j] for i in range(nSeq)]
         for j in range(nSeg) ]
newCarbs = [ [df['NewCarbs'][i][j] for i in range(nSeq)]
         for j in range(nSeg) ] # newCarbs[i] is array of all new-carb arrays at first meas in sequence. 
newInsulin = [ [df['NewInsulin'][i][j] for i in range(nSeq)]
         for j in range(nSeg) ]
delT = [ [df['delT'][i][j] for i in range(nSeq)]
         for j in range(nSeg) ]

# TODO: pull settings from common source (i.e. duration each bin represents)
nBins = len(df['NewCarbs'][0][0])
binSize = 0.5
insActCurve = np.multiply(range(0, nBins), 1./(nBins-1)) # TODO: realistic curves; allow scaling
carbActCurve = np.multiply(range(0, nBins), 1./(nBins-1)) # TODO: realistic curves; allow scaling
timeBinsDown = np.array(list(reversed(np.multiply(range(1, nBins+1), binSize))))

def scaleByTimeBin(amounts, actionCurve):
    nSeq = len(amounts)
    nBins = len(amounts[0]) 
    return [[amounts[j][k] * actionCurve[k] for k in range(nBins)] for j in range(nSeq)]

# Store the insulin on board and carbs on board that have been added at each timepoint, since last measurement
newIOB = [scaleByTimeBin(newInsulin[i], list(reversed(insActCurve))) for i in range(nSeg)]
newCOB = [scaleByTimeBin(newCarbs[i], list(reversed(carbActCurve))) for i in range(nSeg)]

print('newInsulin: {} x {} x {}'.format(len(newInsulin), len(newInsulin[0]), len(newInsulin[0][0])))
print('newIOB: {} x {} x {}'.format(len(newIOB), len(newIOB[0]), len(newIOB[0][0])))
print('delT: {} x {} '.format(len(delT), len(delT[0])))
print(insActCurve)
print(np.minimum(1, 2 * 1/timeBinsDown))
print(nSeq)

In [ ]:
# Compute the current insulin on board, based on how much remaining IOB is present from last measurement
# and what IOB has been added since last measurement.

currentIOB = []
currentIOB.append(np.array(newIOB[0]))

for i in range(1, nSeg):
         
    # How much IOB from previous IOB is still there?
    remainingPreviousIOB = [currentIOB[i-1][j] * np.divide(np.maximum(timeBinsDown - delT[i][j], 0), timeBinsDown) 
                            for j in range(nSeq)]   # nSeq x nBins
    
    # Shift it to keep track of how long each dose has left to act
    binsToShift = np.around(np.divide(delT[i], binSize)).astype('int') # len nBins
    
    shiftedPreviousIOB = [np.append([0] * min(binsToShift[j], nBins), prev[:-min(binsToShift[j], nBins)])
                          for (j, prev) in enumerate(remainingPreviousIOB)]
    
    iob = newIOB[i] + shiftedPreviousIOB
    
    currentIOB.append(iob)
    
print('currentIOB: {} x {} x {}'.format(len(currentIOB), len(currentIOB[0]), len(currentIOB[0][0])))

# Analogously, compute current carbs on board 
# TODO: generalize

currentCOB = []
currentCOB.append(np.array(newCOB[0]))

for i in range(1, nSeg):
         
    # How much IOB from previous IOB is still there?
    remainingPreviousCOB = [currentCOB[i-1][j] * np.divide(np.maximum(timeBinsDown - delT[i][j], 0), timeBinsDown) 
                            for j in range(nSeq)]   # nSeq x nBins
    
    # Shift it to keep track of how long each food has left to act
    binsToShift = np.around(np.divide(delT[i], binSize)).astype('int') # len nBins
    
    shiftedPreviousCOB = [np.append([0] * min(binsToShift[j], nBins), prev[:-min(binsToShift[j], nBins)])
                          #if binsToShift[j] < nBins else [0] * nBins
                          for (j, prev) in enumerate(remainingPreviousCOB)]
    
    cob = newCOB[i] + shiftedPreviousCOB
    
    currentCOB.append(cob)
    
print('currentCOB: {} x {} x {}'.format(len(currentCOB), len(currentCOB[0]), len(currentCOB[0][0])))
    
    
iobImpact = [np.minimum(1, np.outer(delT[i], 1/timeBinsDown)) for i in range(nSeg)]
cobImpact = [np.minimum(1, np.outer(delT[i], 1/timeBinsDown)) for i in range(nSeg)]

In [ ]:
with pm.Model() as model:
    
    # TODO: realistic priors
    #M = pm.Normal('Basal', mu=5, sd=1) # Prior on basal metabolism
    M = pm.Uniform('Basal', lower=0, upper=40)
    CF = pm.Normal('CF', mu=250, sd=50) # Prior on correction factor
    CR = pm.Normal('CR', mu=40, sd=15) # Prior on correction factor
    sigmaBG = pm.Normal('SigmaMeas', mu=10, sd=3)
    
    # iobs = []
    # cobs = []
    glus = []
    glus.append(pm.Normal('glucose0', mu=120, sd=sigmaBG, observed=smbg[0]))

    #iobs.append(pm.Normal('iob0', mu=[0,0,0,0,0,0,0], sd=sigmaIns, observed=newIOB[0]))
    # Initial IOB is just based on new insulin at first timepoint in series.
    #iobs.append(newIOB[0])
    # TODO: infer true IOB, COB
    
    for i in range(1, nSeg):
        print(i)
        # Calculate predicted next glucose value, based on...
        glus.append(pm.Normal('glucose' + str(i), 
                              mu = glus[i-1] + # Last true glucose value
                                   -1 * np.dot(newInsulin[i-1], insActCurve) * CF + # Newly-added insulin
                                   -1 * np.transpose([np.dot(currentIOB[i-1][j], iobImpact[i][j]) for j in range(nSeq)]) * CF +   # All insulin, including new
                                   np.dot(newCarbs[i-1], carbActCurve) * CF / CR + # Newly-added carbs
                                   np.transpose([np.dot(currentCOB[i-1][j], cobImpact[i][j]) for j in range(nSeq)]) * CF / CR + # Carbs on board
                                   (M - basal[i-1]) * delT[i-1]/24 * CF, # Basal/metabolism mismatch
                              sd = sigmaBG, 
                              observed=smbg[i]))

    # TODO: only use values AFTER N hours from start of segment (instead of approx as meas 3) for prediction?
    likelihood = reduce(lambda x, y: x * y, glus)

    # Inference!
    trace = pm.sample(6000, tune=2000, progressbar=True) # draw posterior samples using NUTS sampling
    

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(7, 7))

pm.traceplot(trace[100:])
plt.tight_layout();

TODO: 
* Set up basic tests to (continue to) verify calculations; see if really want to be using jupyter & if there's a better way to use git if so
* Let sigmaBG vary (and look into using lognormal distribution) - does this matter for assessing certainty?
* Breakfast/lunch/dinner ratios
* Use more realistic priors
* Use actual Humalog action time profiles
* Allow action time to scale
* Appropriate noise on insulin, carbs and infer "true" IOB/COB profiles
* Verify indexing
* Basal profile
* Import pump basal
* Treat sugar to treat differently
* Treat exercise snacks differently
* Allow complete discounting of a timepoint with some small probability - the "hrmmmm" factor for typos, REALLY off measurements, etc.


* Evaluate model fit to data
* Conduct comparisons to actual protocols
* Display predictions
* Code cleanup for readability
* Code cleanup for efficiency
* Set up to import other datasets